Goal of this notebook:
 - Implement a way of representing types by having an edge going into a vertex that holds a type
 - Implement shortest path queries that avoid vertices connected to a specific vertex of a particular type

In [58]:
# Jupyter notebook needs this or else it will crash
import nest_asyncio
nest_asyncio.apply()

from gremlin_python import statics
from gremlin_python.structure.graph import Graph
from gremlin_python.process.graph_traversal import __
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.process.traversal import P # NEW!!! Import predicates (gt, gte, lt, lte, etc.)
from gremlin_python.process.traversal import Cardinality # NEW!!! Import Cardinality such as list_, set_ and single.
from gremlin_python.driver.protocol import GremlinServerError # Gremlin server error
from gremlin_python.process.traversal import Pop # for Pop.all_ in select(Pop.all_, 'v')

# Instantiate a Gremlin Graph
graph = Graph()

# Connect to the server, instantiate traversal of graph.
g = graph.traversal().withRemote(DriverRemoteConnection('ws://localhost:8182/gremlin','g'))

# Get the vertices of the graph as a list, and print them.

print(g.V().toList())

[v[557096], v[418032], v[422128], v[41390216], v[421920], v[446632]]


In [59]:
def connected(name1: str, name2: str, time: float) -> bool:
    """
    Given two vertices labelled with <name1> and <name2>, determine whether they were connected at time <time>. 
    Do so by sending a Gremlin query to determine whether there exists an edge between the 
    two vertices such that <time> falls in between their "start_time" and "end_time" parameters.
    TODO: Add sphinx documentation if this will be implemented into the actual Python library.
    """

    # Get the vertices associated with the names
    # v1, v2 = g.V().has('name', name1).next(), g.V().has('name', name2).next()

    # Return whether there are edges that:
    #   - connect v1 and v2, 
    #   - labelled 'connection',
    #   - have a start time that is less than or equal to <time>
    #   - either do not have an end time or have an end time that is greater than or equal to <time>
    return g.V().has('name', name1).bothE('connection').as_('e').bothV().has('name', name2).select('e').and_(
            __.has('start', P.lte(time)),   # want start time to be less than or equal to <time>
            __.or_(
                __.hasNot('end'),           # end time doesn't have to exist 
                __.has('end', P.gt(time))  # OR end time must be greater than <time>
            )
        ).count().next() > 0

In [60]:
def set_connection(name1: str, name2: str, time: float, connection: bool) -> None:
    """
    Given two vertices labelled with <name1> and <name2>, create a new connection or terminate their existing connection, based on the value of <bool>. Label with time <time>.

    TODO: Add sphinx documentation if this will be implemented into the actual Python library.
    """

    if connection:
        if not connected(name1, name2, time):

            # Add an edge labelled 'connection' with a start time of <time>
            g.V().has('name', name1).as_("a").V().has('name', name2).as_("b").addE('connection').from_("a").to("b").property('start', time).iterate()

    else:
        # For all edges between v1 and v2 labelled 'connection' (there should only be one) that do not have an 'end' property, create an end property of <time>.
        g.V().has('name', name1).bothE('connection').as_('e').bothV().has('name', name2).select('e').hasNot('end').property('end', time).iterate()

In [61]:
def set_type(name: str, type: str) -> None:
    """
    Connect the vertex labelled with <name> to a vertex labelled <type> with an edge going into the <type> vertex labelled with "type".

    TODO: Add sphinx documentation if this will be implemented into the actual Python library.
    """

    g.V().has('name', name).as_("a").V().has('name', type).as_("b").addE("type").from_("a").to("b").iterate()

In [62]:
# This will put it in V-E-V-E-V-...-V form as a list per path.
def find_paths(name1: str, name2: str, avoid_type: str, time: float):
    """
    Given two vertices labelled with <name1> and <name2>, return the paths that connect the vertices by edges that were active at <time> as a list.

    Avoid vertices of type avoid_type.

    TODO: Add sphinx documentation if this will be implemented into the actual Python library.
    """

    return g.V().has('name', name1).repeat(
            __.bothE().and_(
                __.has('start', P.lte(time)),   # want start time to be less than or equal to <time>
                __.or_(
                    __.hasNot('end'),           # end time doesn't have to exist 
                    __.has('end', P.gt(time))  # OR end time must be greater than <time>
                )
            ).otherV().not_(__.outE('type').inV().has('name', avoid_type)).simplePath()
        ).until(__.has('name', name2)).path().toList()

In [63]:
# Drop all vertices.
g.V().drop().iterate()

g.addV().property('name', 'TYPE1').iterate()
g.addV().property('name', 'TYPE2').iterate()

for i in range(1, 5):
    print(f"v{i}", g.addV().property('name', f'v{i}').next())

set_type('v1', 'TYPE1')
set_type('v2', 'TYPE1')
set_type('v3', 'TYPE1')
set_type('v4', 'TYPE2')

# Connect v1 and v2 at time 1. Disconnect at time 10.
set_connection('v1', 'v2', 1, True)
set_connection('v1', 'v2', 10, False)

# Connect v1 and v3 at time 2, disconnect at time 4.
set_connection('v1', 'v3', 2, True)
set_connection('v1', 'v3', 5, False)

# Connect v2 and v3 at time 3, disconnect at time 7.
set_connection('v2', 'v3', 3, True)
set_connection('v3', 'v3', 7, False)

# Connect v1 and v4 at time 1, disconnect at time 7.
set_connection('v1', 'v4', 1, True)
set_connection('v1', 'v4', 7, False)

# Connect v4 and v2 at time 2, disconnect at time 9.
set_connection('v4', 'v2', 2, True)
set_connection('v4', 'v2', 9, False)

# EXPORT THE GRAPH.
g.io("avoid_vertices_graph.xml").write().iterate()

for time in range(0, 12):
    print(f"///// {time} /////")
    for arr in find_paths(name1='v1', name2='v2', avoid_type='TYPE2', time=time):
        print(list(arr))
    print()

v1 v[454720]
v2 v[41455800]
v3 v[884832]
v4 v[41394312]
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fbcc817a610>
Exception ignored in: <function ClientResponse.__del__ at 0x7fbcc8d37b80>
Traceback (most recent call last):
  File "/home/azszavyalov/.local/lib/python3.8/site-packages/aiohttp/client_reqrep.py", line 803, in __del__
    self._connection.release()
  File "/home/azszavyalov/.local/lib/python3.8/site-packages/aiohttp/connector.py", line 170, in release
    self._connector._release(
  File "/home/azszavyalov/.local/lib/python3.8/site-packages/aiohttp/connector.py", line 650, in _release
    protocol.close()
  File "/home/azszavyalov/.local/lib/python3.8/site-packages/aiohttp/client_proto.py", line 63, in close
    transport.close()
  File "/usr/lib/python3.8/asyncio/selector_events.py", line 690, in close
    self._loop.call_soon(self._call_connection_lost, None)
  File "/usr/lib/python3.8/asyncio/base_events.py", line 719, in call_soon
 